In [2]:
import pandas as pd
import numpy as np
import datetime
import folium
import math
import os
import re
import pymysql
from collections import namedtuple
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [13]:
# 메모리 문제로 월단위로 데이터 추출 
set_date = '%2021-07%'

In [14]:
# 배달기사위치정보

conn  = pymysql.connect(host ='121.165.19.66', port = 3306, user = 'kk-bigdata',
                       password = 'Kgu0318885100!', db = 'kt_upload_20210801',charset = 'utf8')

cursor = conn.cursor()

sql = """select * from kt_upload_20210801.KGU_DLVR_MAN_LC_INFO WHERE LCINFO_REGIST_DT LIKE '%s';"""%set_date
cursor.execute(sql)
rows = cursor.fetchall()

row_list = []
for row in rows:
    row_list.append(row)
    
data = pd.DataFrame(row_list)
data.columns = ['SN', 'LCINFO_REGIST_DT', 'DLVR_MAN_ID', 'DLVR_ENTRPS_ID', 'LO_XCRD', 'LA_YCRD']

SyntaxError: invalid syntax (<ipython-input-14-29eab7937021>, line 4)

In [8]:
data = data[(data['LO_XCRD'] !=0) & (data['LA_YCRD'] !=0)]
data = data.dropna(axis = 0, how = 'any', subset = ['LO_XCRD','LA_YCRD'])
data.shape

(508381, 6)

In [9]:
data['datetime'] = data['LCINFO_REGIST_DT'].apply(lambda x: str(x)[:13])

In [10]:
# 배달기사별 / 시간순서별로 정렬
data = data.sort_values(by =['LCINFO_REGIST_DT','DLVR_MAN_ID'])

In [11]:
#배달기사 동선 & 방향 화살표 표시
def get_bearing(p1, p2):
    
    '''
    Returns compass bearing from p1 to p2
    
    Parameters
    p1 : namedtuple with lat lon
    p2 : namedtuple with lat lon
    
    Return
    compass bearing of type float
    
    Notes
    Based on https://gist.github.com/jeromer/2005586
    '''
    
    long_diff = np.radians(float(p2.lon) - float(p1.lon) )
    
    lat1 = np.radians(float(p1.lat))
    lat2 = np.radians(float(p2.lat))
    
    x = np.sin(long_diff) * np.cos(lat2)
    y = (np.cos(lat1) * np.sin(lat2) 
        - (np.sin(lat1) * np.cos(lat2) 
        * np.cos(long_diff)))
    bearing = np.degrees(np.arctan2(x, y))
    
    # adjusting for compass bearing
    if bearing < 0:
        return bearing + 360
    return bearing


def get_arrows(locations, color=1, size=6, n_arrows=3):
    
    '''
    Get a list of correctly placed and rotated 
    arrows/markers to be plotted
    
    Parameters
    locations : list of lists of lat lons that represent the 
                start and end of the line. 
                eg [[41.1132, -96.1993],[41.3810, -95.8021]]
    arrow_color : default is 'blue'
    size : default is 6
    n_arrows : number of arrows to create.  default is 3
    Return
    list of arrows/markers
    '''
    
    Point = namedtuple('Point', field_names=['lat', 'lon'])
    
    # creating point from our Point named tuple
    p1 = Point(locations[0][0], locations[0][1])
    p2 = Point(locations[1][0], locations[1][1])
    
    # getting the rotation needed for our marker.  
    # Subtracting 90 to account for the marker's orientation
    # of due East(get_bearing returns North)
    rotation = get_bearing(p1, p2) - 90
    
    # get an evenly space list of lats and lons for our arrows
    # note that I'm discarding the first and last for aesthetics
    # as I'm using markers to denote the start and end
    arrow_lats = np.linspace(float(p1.lat), float(p2.lat), n_arrows + 2)[1:n_arrows+1]
    arrow_lons = np.linspace(float(p1.lon), float(p2.lon), n_arrows + 2)[1:n_arrows+1]
    
    arrows = []
    
    #creating each "arrow" and appending them to our arrows list
    for points in zip(arrow_lats, arrow_lons):
        arrows.append(folium.RegularPolygonMarker(location=points, 
                      fill_color=color, number_of_sides=3, 
                      radius=size, rotation=rotation))
    return arrows

In [12]:
def deliman_hour_mapping(select_data,year,month,day):
    hours = [str(hour).zfill(2) for hour in range(0, 25)]  # ['01','02'...'24'] 생성하는 리스트 시간비교하기위해
    deli_man_list = list(set(select_data['DLVR_MAN_ID']))  # 기사별리스트    
    for deliman in deli_man_list:  # 기사리스트에서 기사한명을 뽑아
        part_data = select_data[(select_data['DLVR_MAN_ID'] == deliman)]  # 데이터프레임하나를 만든다
        
        for hour in hours:  # 기사한명 데이터에서 시간을 비교하며 반복하고 일치하면 진행
            man_xy_list = []            
            for i in range(len(part_data)):  # 기사한명이 가지고 있는 데이터 갯수만큼 반복한다
                if part_data.iloc[i]['datetime'] == (year+'-'+month+'-'+day+' '+hour):  # 기사한명의 가지고 있는 datetime이 hour반복된 시간과 같으면
                    lon = part_data.iloc[i]['LO_XCRD']  # 즉, hour가가지고 있는시간 ex) 16 과 기사가 배달한 시간의 시가 같으면or존재하면
                    lat = part_data.iloc[i]['LA_YCRD']
                    lon = float(lon)
                    lat = float(lat)
                    lonlat = [lat, lon]
                    man_xy_list.append(lonlat)  # 배달기사의 한시간 좌표를 저장해놓은 리스트

                    
            if len(man_xy_list) >= 2:  # 만약 데이터가 존재한다면           
                lon_list = []
                lat_list = []
                for i in range(len(man_xy_list)):
                    lon_list.append(man_xy_list[i][0])
                    lat_list.append(man_xy_list[i][1])
                    lon_min = min(lon_list)
                    lat_min = min(lat_list)
                    lon_max = max(lon_list)
                    lat_max = max(lat_list)
                    mean_lon = ((lon_min+lon_max)/2)
                    mean_lat = ((lat_min+lat_max)/2)
                map_osm = folium.Map(location=[mean_lon,mean_lat],zoom_start=15.3,control_scale=True)
                for i in man_xy_list:  # 좌표 marker를 생성하라
                    folium.Marker(i).add_to(map_osm)
                arrows_list = []
                for i in range(1, len(man_xy_list)):
                    folium.PolyLine(locations=[man_xy_list[i - 1], man_xy_list[i]], weigts=0.05).add_to(map_osm)
                    arrows = get_arrows(locations=[man_xy_list[i - 1], man_xy_list[i]], size=4, n_arrows=2)
                    arrows_list.append(arrows)
                for arrow in arrows_list:
                    for i in range(len(arrow)):
                        arrow[i].add_to(map_osm)
             
                   # html파일로 저장

                if (lon_max - lon_min) < 0.0293 and (lon_max - lon_min) != 0:
                    map_osm.save("C:/Users/inthe/Documents/배달기사이미지데이터/html/%s_%s_%s_%s_%s.html"%(year,month,day,hour,deliman))

    
def select_date(data,year,month,day):
    data['year']= data['datetime'].apply(lambda x: str(x)[:4])
    data['month'] = data['datetime'].apply(lambda x : str(x)[5:7])
    data['day'] = data['datetime'].apply(lambda x: str(x)[8:10])
    df = data[(data['year'] == year) & (data['month'] == month) & (data['day'] == day)]
    return df


def run_mapping(data,year,month,day):
    select_data = select_date(data,year,month,day)
    deliman_hour_mapping(select_data,year,month,day)

In [ ]:
import time
start = time.time()
days = [ str(day).zfill(2) for day in range(1,32)]
print(days)
for day in days:
    print(day)
    run_mapping(data,'2021','07',day)
print(time.time() - start)


['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31']
01
02
03
04
05
06
07
08
09
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30


In [ ]:
def html_to_png(dirname):
    file_list = os.listdir(dirname)
    html_list = [file for file in file_list if file.endswith(".html")]
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('headless')
#     chrome_options.add_argument('--disable-gpu')
    chrome_options.add_argument('lang=ko_KR')

    driver = webdriver.Chrome('chromedriver.exe',chrome_options=chrome_options)
    driver.set_window_size(1280,900)

    for html in html_list:
        driver.get("C:/Users/inthe/Documents/배달기사이미지데이터/html/%s"%html)
        screenshot_name = html[:-5]+".jpeg"
        driver.save_screenshot("C:/Users/inthe/Documents/배달기사이미지데이터/KGU_KGUDLIVERYLCIMG00001_20210731000000/"+screenshot_name)

In [ ]:
import time
start = time.time()
html_to_png("C:/Users/inthe/Documents/배달기사이미지데이터/html/")
print(time.time() - start)